## River figure(s): cumulative flow, weekly or bi-monthly flow

### Import modules and files

In [339]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import datetime
import plotly.express as px
import plotly.graph_objects as go
from dateutil import rrule, parser

In [265]:
%cd '/home/lindsay/'

heeia_13 = pd.read_csv('heeia_13.csv', sep='\t', header=None)
heeia_13.columns = ['USGS', 'ID', 'Date', 'Zone', 'Heeia', 'A']
heeia_13 = pd.DataFrame.drop(heeia_13, columns=['USGS', 'ID', 'Zone', 'A'])
heeia_14 = pd.read_csv('heeia_14.csv', sep='\t', header=None)
heeia_14.columns = ['USGS', 'ID', 'Date', 'Zone', 'Heeia', 'A']
heeia_14 = pd.DataFrame.drop(heeia_14, columns=['USGS', 'ID', 'Zone', 'A'])
waiahole_13 = pd.read_csv('waiahole_13.csv', sep='\t', header=None) 
waiahole_13.columns = ['USGS', 'ID', 'Date', 'Zone', 'Waiahole', 'A']
waiahole_13 = pd.DataFrame.drop(waiahole_13, columns=['USGS', 'ID', 'Zone', 'A'])
waiahole_14 = pd.read_csv('waiahole_14.csv', sep='\t', header=None)
waiahole_14.columns = ['USGS', 'ID', 'Date', 'Zone', 'Waiahole', 'A']
waiahole_14 = pd.DataFrame.drop(waiahole_14, columns=['USGS', 'ID', 'Zone', 'A'])
waihee_13 = pd.read_csv('waihee_13.csv', sep='\t', header=None) 
waihee_13.columns = ['USGS', 'ID', 'Date', 'Zone', 'Waihee', 'A']
waihee_13 = pd.DataFrame.drop(waihee_13, columns=['USGS', 'ID', 'Zone', 'A'])                                
waihee_14 = pd.read_csv('waihee_14.csv', sep='\t', header=None) 
waihee_14.columns = ['USGS', 'ID', 'Date', 'Zone', 'Waihee', 'A']
waihee_14 = pd.DataFrame.drop(waihee_14, columns=['USGS', 'ID', 'Zone', 'A']) 
waikane_13 = pd.read_csv('waikane_13.csv', sep='\t', header=None)
waikane_13.columns = ['USGS', 'ID', 'Date', 'Zone', 'Waikane', 'A']
waikane_13 = pd.DataFrame.drop(waikane_13, columns=['USGS', 'ID', 'Zone', 'A']) 
waikane_14 = pd.read_csv('waikane_14.csv', sep='\t', header=None) 
waikane_14.columns = ['USGS', 'ID', 'Date', 'Zone', 'Waikane', 'A']
waikane_14 = pd.DataFrame.drop(waikane_14, columns=['USGS', 'ID', 'Zone', 'A']) 

/home/lindsay


### Data cleaning

Given the inconsistent data lengths and sporadic missing data, I'm going to start by merging all CSVs based on the Date column.

Next, I'll separate the timestamps from the dates, then resample to mean daily values, calculate cumulative running sums, and finally, resample to mean weekly values.

In [266]:
df_13 = pd.merge(waiahole_13, waikane_13, on='Date', how='outer', sort = True)
df_13 = pd.merge(df_13, waihee_13, on='Date', how='outer', sort = True)
df_13 = pd.merge(df_13, heeia_13, on='Date', how='outer', sort = True)

df_14 = pd.merge(waiahole_14, waikane_14, on='Date', how='outer', sort = True)
df_14 = pd.merge(df_14, waihee_14, on='Date', how='outer', sort = True)
df_14 = pd.merge(df_14, heeia_14, on='Date', how='outer', sort = True)

In [267]:
# Remove timestamps
df_13['Date'] = pd.to_datetime(df_13['Date']).dt.date
df_14['Date'] = pd.to_datetime(df_14['Date']).dt.date
# Classify date col as datetime object
df_13['Date'] = pd.to_datetime(df_13['Date'])
df_14['Date'] = pd.to_datetime(df_14['Date'])
# Group by and average values by day
df_13 = df_13.set_index('Date').groupby(pd.Grouper(freq='d')).mean().dropna(how='all')
df_14 = df_14.set_index('Date').groupby(pd.Grouper(freq='d')).mean().dropna(how='all')

In [268]:
# Calculate cumulative sums
df_13['Waiahole_cs'] = df_13['Waiahole'].cumsum()
df_13['Waikane_cs'] = df_13['Waikane'].cumsum()
df_13['Waihee_cs'] = df_13['Waihee'].cumsum()
df_13['Heeia_cs'] = df_13['Heeia'].cumsum()

df_14['Waiahole_cs'] = df_14['Waiahole'].cumsum()
df_14['Waikane_cs'] = df_14['Waikane'].cumsum()
df_14['Waihee_cs'] = df_14['Waihee'].cumsum()
df_14['Heeia_cs'] = df_14['Heeia'].cumsum()

In [427]:
# Weekly resampling
df_13_weekly = df_13.resample('1W').mean().dropna(how='all')
df_14_weekly = df_14.resample('1W').mean().dropna(how='all')
# Sum all four cumulative sums 
df_13_weekly['Cumulative'] = df_13_weekly.iloc[:,4:7].sum(axis=1)
df_14_weekly['Cumulative'] = df_14_weekly.iloc[:,4:7].sum(axis=1)
# Add weekly x column, 'Date'
df_13_weekly['Date'] = df_13_weekly.index
df_14_weekly['Date'] = df_14_weekly.index
# Drop individual river cumulative sums 
df_13_weekly = pd.DataFrame.drop(df_13_weekly, columns=['Waiahole_cs', 'Waikane_cs', 'Waihee_cs', 'Heeia_cs']) 
df_14_weekly = pd.DataFrame.drop(df_14_weekly, columns=['Waiahole_cs', 'Waikane_cs', 'Waihee_cs', 'Heeia_cs']) 

In [428]:
# Change units from cubic ft per s to L per s
df_13_weekly['Waiahole'] = df_13_weekly['Waiahole'].apply(lambda x: x*28.3168)
df_13_weekly['Waikane'] = df_13_weekly['Waikane'].apply(lambda x: x*28.3168)
df_13_weekly['Waihee'] = df_13_weekly['Waihee'].apply(lambda x: x*28.3168)
df_13_weekly['Heeia'] = df_13_weekly['Heeia'].apply(lambda x: x*28.3168)
df_13_weekly['Cumulative'] = df_13_weekly['Cumulative'].apply(lambda x: x*28.3168)

df_14_weekly['Waiahole'] = df_14_weekly['Waiahole'].apply(lambda x: x*28.3168)
df_14_weekly['Waikane'] = df_14_weekly['Waikane'].apply(lambda x: x*28.3168)
df_14_weekly['Waihee'] = df_14_weekly['Waihee'].apply(lambda x: x*28.3168)
df_14_weekly['Heeia'] = df_14_weekly['Heeia'].apply(lambda x: x*28.3168)
df_14_weekly['Cumulative'] = df_14_weekly['Cumulative'].apply(lambda x: x*28.3168)

In [429]:
# Calculate mean discharge for all rivers
df_13_weekly['Average'] = df_13_weekly.iloc[:,0:4].mean(axis=1)
df_14_weekly['Average'] = df_14_weekly.iloc[:,0:4].mean(axis=1)

In [430]:
pd.DataFrame.to_csv(df_13_weekly, 'df_13_weekly.csv')
pd.DataFrame.to_csv(df_14_weekly, 'df_14_weekly.csv')

In [365]:
# Tired, manual editing ahead, insert caution sirens.
df = pd.read_csv('df_weekly.csv')

In [382]:
df_13_weekly['Month'] = df['Month'].values.astype(str)
df_14_weekly['Month'] = df['Month'].values.astype(str)

### Plot party

For each year, I want to show the weekly discharge, and I also want to plot the running summed total for the year. 

In [418]:
frames = [df_13_weekly, df_14_weekly]
df_all = pd.concat(frames)
len(df_all)

54

In [392]:
df_all.head()

,Waiahole,Waikane,Waihee,Heeia,Cumulative,Date,Average,Month
Date,,,,,,,,
2013-01-06,888.368808,690.200714,NaN,45.505688,5143.494083,2013-01-06,541.358403,Jan
2013-01-13,809.616079,345.567777,NaN,41.187881,12668.963466,2013-01-13,398.790579,Jan
2013-01-20,755.970070,257.327234,NaN,44.657532,20290.507721,2013-01-20,352.651612,Jan
2013-01-27,730.324825,304.974043,NaN,54.102365,26822.016564,2013-01-27,363.133744,Jan
2013-02-03,1191.412505,511.000675,110.000936,56.413639,39205.858611,2013-02-03,467.206939,Feb


In [421]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                x=df_all.Date,
                y=df_all['Average'],
                name="Average",
                line_color='deepskyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=df_all.Date,
                y=df_all['Cumulative'],
                name="Cumulative",
                line_color='deepskyblue',
                opacity=0.8))

# Use date string to set xaxis range
fig.update_layout(xaxis_range=['2013-01-02','2014-12-29'],
                  title_text="Weekly mean river discharge into Kaneohe Bay",
                  xaxis_title="Date",
                  yaxis_title="Discharge (L/s)")
fig.show()